In [ ]:
import matplotlib.pyplot as plt
import csv
import sklearn
import pickle
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import httplib2
from urlextract import URLExtract
import nltk
nltk.download("punkt")
import warnings
warnings.filterwarnings('ignore')
import re
import math
import string
nltk.download('stopwords')
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import json 

In [ ]:
data = pd.read_csv('/content/smishing_dataset.csv', encoding='latin-1')

In [ ]:
data["Url"] = data.apply(lambda _: [], axis=1)

def check_url(url):
    http = httplib2.Http(timeout=0.1)
    if "http://" in url or "https://" in url:
        try:
            mod_url = url
            resp = http.request(mod_url, 'HEAD')
            check_flag = int(resp[0]['status']) < 400
            return check_flag
        except:
            return False
    else:
        try:
            mod_url = "http://" + url
            resp = http.request(mod_url, 'HEAD')
            check_flag = int(resp[0]['status']) < 400
            return check_flag 
        except:
            try:
                mod_url = "https://" + url
                resp = http.request(mod_url, 'HEAD')
                check_flag = int(resp[0]['status']) < 400
                return check_flag 
            except:
                return False
def find_url(text):
    extractor = URLExtract()
    urls = extractor.find_urls(text)
    for url in urls:
       text = text.replace(url, "")
    return urls
data['Url'] = data['Message'].apply(find_url)

for index, row in data.iterrows():
    for url in row['Url']:
        row['Message'] = row['Message'].replace(url,"")


In [ ]:
data.to_csv("smishing_data_clean.csv", index=False)

In [ ]:
data = pd.read_csv('/content/smishing_data_clean.csv', encoding='latin-1')
ham_words = {}
spam_words = {}
total_spam_words = 0
total_ham_words = 0
total_words = 0
words_bucket ={}

In [ ]:
from tkinter.constants import W
for val in data[data['Class'] == 'smish'].Message:
    text = val.lower()
    tokens = nltk.word_tokenize(text)
    for words in tokens:
        if words not in spam_words:
            spam_words[words] = 0
        spam_words[words] += 1
        total_spam_words += 1
        total_words += 1
        words_bucket[words] = 1

# Creating a corpus of ham messages
for val in data[data['Class'] == 'ham'].Message:
    text = val.lower()
    tokens = nltk.word_tokenize(text)
    for words in tokens:
        if words not in ham_words:
            ham_words[words] = 0
        ham_words[words] += 1
        total_ham_words += 1
        total_words += 1
        words_bucket[words] = 1

In [ ]:
data = data.replace(['ham','smish'],[0, 1])

In [ ]:
for word in words_bucket:
    if(word in stopwords.words('english') or word in list(string.punctuation)):
        if(word in spam_words):
            total_words -= spam_words[word]
            spam_words.pop(word)
        if(word in ham_words):
            total_words -= ham_words[word]
            ham_words.pop(word)
for word in spam_words:
    spam_words[word] = math.log(spam_words[word]/total_words)

In [ ]:
def text_process(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english') and list(string.punctuation)]
    return " ".join(text)
data['Message'] = data['Message'].apply(text_process)

In [ ]:
text = pd.DataFrame(data['Message'])
label = pd.DataFrame(data['Class'])
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data['Message'])
features = vectors
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.15, random_state=111)

In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)

In [ ]:
clfs = {'SVC' : svc,'KN' : knc, 'NB': mnb, 'DT': dtc, 'LR': lrc, 'RF': rfc}
def train(clf, features, targets):
    clf.fit(features, targets)

def predict(clf, features):
    return (clf.predict(features))

In [ ]:
pred_scores_word_vectors = []
for k,v in clfs.items():
    train(v, X_train, y_train)
    pred = predict(v, X_test)
    pred_scores_word_vectors.append((k, [accuracy_score(y_test , pred)]))

In [ ]:
pred_scores_word_vectors

In [ ]:
mnb = MultinomialNB(alpha=0.2)
mnb.fit(X_train,y_train)
pred = mnb.predict(X_test)
y_pred_nb = mnb.predict(X_test)
y_true_nb = y_test
cm = confusion_matrix(y_true_nb, y_pred_nb)
f, ax = plt.subplots(figsize =(5,5))
sns.heatmap(cm,annot = True,linewidths=0.5,linecolor="red",fmt = ".0f",ax=ax)
plt.xlabel("y_pred_nb")
plt.ylabel("y_true_nb")
plt.show()

In [ ]:
with open("words.json", "w") as outfile:
    json.dump(spam_words, outfile)